In [9]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np

In [10]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

In [11]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [12]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [13]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

In [14]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [15]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [16]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones')(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [17]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [18]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [19]:
y_train_agreement.shape

(25000, 83)

In [20]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=10)

Train on 16667 samples, validate on 8333 samples
Epoch 1/10
16667/16667 [==============================] - 21s 1ms/step - loss: 0.4555 - acc: 0.6984 - val_loss: 0.4419 - val_acc: 0.7605
Epoch 2/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.3216 - acc: 0.8279 - val_loss: 0.4501 - val_acc: 0.7854
Epoch 3/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.2791 - acc: 0.8584 - val_loss: 0.4275 - val_acc: 0.8111
Epoch 4/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.2515 - acc: 0.8775 - val_loss: 0.4473 - val_acc: 0.8158
Epoch 5/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.2366 - acc: 0.8876 - val_loss: 0.4460 - val_acc: 0.8213
Epoch 6/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.2202 - acc: 0.8994 - val_loss: 0.4492 - val_acc: 0.8213
Epoch 7/10
16667/16667 [==============================] - 20s 1ms/step - loss: 0.2074 - acc: 0.9057 - val_loss: 0.4559 - val_acc:

In [21]:
combined_model_history = combined_model.fit([X_train,y_train_agreement], y_train_original, batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 89s 4ms/step - loss: 0.7342 - acc: 0.3490
Epoch 2/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6788 - acc: 0.3810
Epoch 3/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6795 - acc: 0.3761
Epoch 4/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6771 - acc: 0.3784
Epoch 5/10
25000/25000 [==============================] - 86s 3ms/step - loss: 0.6800 - acc: 0.3738
Epoch 6/10
25000/25000 [==============================] - 86s 3ms/step - loss: 0.6807 - acc: 0.3776
Epoch 7/10
25000/25000 [==============================] - 86s 3ms/step - loss: 0.6777 - acc: 0.3786
Epoch 8/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6773 - acc: 0.3757
Epoch 9/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6818 - acc: 0.3802
Epoch 10/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.6799 - acc: 0.3769

In [22]:
# independent weight here
# independent_weight_base_model_history
# independent_weight_combined_model_history

In [23]:
# var name
# shared_weight_base_model_history
# shared_weight_combined_model_history

In [24]:
net_weight = base_model.get_weights()

In [25]:
len(net_weight)

2

In [26]:
weight = net_weight[0].flatten()

In [27]:
net_weight[1].shape

(1,)

In [28]:
words = cv.get_feature_names()

In [29]:
indices = np.argsort(weight)

In [30]:
for i in indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

worst 	 -3.176
incoherent 	 -2.748
unwatchable 	 -2.727
mst3k 	 -2.481
dull 	 -2.419
waste 	 -2.398
awful 	 -2.371
poorly 	 -2.346
fails 	 -2.322
pointless 	 -2.307


In [31]:
pos_indices = indices[::-1]
for i in pos_indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

refreshing 	 3.012
excellent 	 2.838
touching 	 2.781
rare 	 2.448
wonderfully 	 2.369
funniest 	 2.367
complaint 	 2.358
definitely 	 2.275
flawless 	 2.241
underrated 	 2.226


In [32]:
combined_net_weight = combined_model.get_weights()

In [33]:
len(combined_net_weight)

87

In [34]:
ht_weight = []
for i in range(len(combined_net_weight)):
    if i>1 and i < 85:
        print(combined_net_weight[i].flatten())
        ht_weight.append(combined_net_weight[i].flatten())
    else:
        print(i, combined_net_weight[i].shape)
ht_weight = np.asarray(ht_weight)

0 (3641, 1)
1 (1,)
[3.6556344]
[3.7559261]
[4.102407]
[4.5520864]
[2.4554942]
[1.1367724]
[3.9148142]
[4.1943164]
[3.4597213]
[3.9281178]
[2.9604053]
[3.886339]
[3.9110675]
[4.2697277]
[3.735556]
[2.468304]
[2.9559186]
[2.468173]
[2.8160737]
[3.6252472]
[3.6582925]
[3.431016]
[3.8260071]
[3.781578]
[4.0455384]
[4.0051327]
[4.1663094]
[4.191593]
[3.0170162]
[2.816549]
[3.0181923]
[3.9180963]
[2.997084]
[3.7094884]
[3.1330636]
[3.7126858]
[2.5833817]
[3.2916634]
[2.7945833]
[2.8794441]
[2.7706103]
[4.119616]
[3.510894]
[4.3403544]
[4.0395174]
[3.7672327]
[3.8554194]
[3.6374571]
[2.7802176]
[4.2707977]
[3.449505]
[3.0845454]
[3.5340755]
[3.5334406]
[3.9833667]
[2.8250175]
[2.0635028]
[4.2084146]
[3.8613415]
[3.0394914]
[3.4362292]
[3.250239]
[3.4422114]
[3.3563075]
[3.604483]
[4.1225843]
[3.651818]
[2.96045]
[3.8483667]
[3.3175228]
[3.4998188]
[3.140967]
[4.2600293]
[4.078099]
[3.659268]
[3.1592612]
[4.2043757]
[2.7366154]
[3.6606736]
[3.2636013]
[1.6055597]
[3.9620974]
[4.341918]
85 (83,

In [35]:
sigmoid_weight = combined_net_weight[85].flatten()

In [36]:
ht_weight = ht_weight.flatten()

In [37]:
human_term_indices = np.argsort(ht_weight)

print('human terms \t dense_relu weight \t sigmoid weight \t\t pos/neg?')
for i in human_term_indices:
    print('%s \t %.3f \t %.3f \t\t %.1f' %(word_list[i], ht_weight[i], sigmoid_weight[i], connotation[word_list[i]]))

human terms 	 dense_relu weight 	 sigmoid weight 		 pos/neg?
6/10 	 1.137 	 0.227 		 1.0
wonderfully 	 1.606 	 0.698 		 1.0
perfectly 	 2.064 	 1.233 		 1.0
5/10 	 2.455 	 1.487 		 1.0
beautifully 	 2.468 	 1.498 		 1.0
badly 	 2.468 	 -1.283 		 0.0
fun 	 2.583 	 1.800 		 1.0
wasted 	 2.737 	 -1.692 		 0.0
great 	 2.771 	 1.550 		 1.0
loved 	 2.780 	 1.762 		 1.0
funniest 	 2.795 	 1.892 		 1.0
best 	 2.816 	 1.661 		 1.0
enjoyed 	 2.817 	 1.907 		 1.0
perfect 	 2.825 	 2.012 		 1.0
gem 	 2.879 	 1.916 		 1.0
beautiful 	 2.956 	 1.827 		 1.0
amazing 	 2.960 	 2.026 		 1.0
solid 	 2.960 	 2.070 		 1.0
fantastic 	 2.997 	 1.974 		 1.0
enjoyable 	 3.017 	 2.062 		 1.0
excellent 	 3.018 	 1.955 		 1.0
poorly 	 3.039 	 -2.045 		 0.0
mst3k 	 3.085 	 -2.282 		 0.0
favorite 	 3.133 	 1.965 		 1.0
surprisingly 	 3.141 	 2.352 		 1.0
unfunny 	 3.159 	 -2.322 		 0.0
rare 	 3.250 	 2.288 		 1.0
wonderful 	 3.264 	 2.046 		 1.0
funny 	 3.292 	 -2.251 		 0.0
subtle 	 3.318 	 2.366 		 1.0
redeeming 	

In [38]:
len(connotation)

83

In [39]:
# Print report on the word transparency
index = [9, 19]
def report():
    for i in index:
        print()
        bm = base_model.predict(X_test[i])
        
        cm = combined_model.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
        print(X_test_original[i], '\n\n actual label : ', y_test_original[i], '\n predict from base model : ', bm.flatten(), '\n predict label : ', cm.flatten())
    
        for i,output in enumerate(concat_output.flatten()):
            if output != 0:
                print(word_list[i], output)

In [40]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [41]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [42]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)

25000/25000 [==============================] - 2s 62us/step


In [43]:
score

[0.1866892215681076, 0.79436]

### Let's see the weight when the trainable is false

In [44]:
def build_combined_model():
    # build the combined model
    # Combined model
    human_terms_len = len(word_list)

    base_model = build_base_model(X_train.shape[1])

    combined_input_layer = Input(shape=(X_train.shape[1],))

    # build the hard coded weight for human terms
    ht_input_layer = Input(shape=(human_terms_len,))

    split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

    # get the document prediction
    label_layer = base_model(combined_input_layer)

    # stack the multiply layer
    dense_layer = []
    for i in range(human_terms_len):
        dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones', trainable=True)(Multiply()([split[i], label_layer])))

    # concat all the result   
    concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

    # pass it to sigmoid layer
    output_layer = Dense(1, activation='sigmoid')(concat)

    combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
    combined_model.summary()
    
    return base_model, combined_model

In [45]:
base_model, combined_model = build_combined_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [46]:
base_model.compile(loss='mse',
                  optimizer='Adagrad',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='Adagrad',
                      metrics=['acc'])

In [47]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=50)

Train on 16667 samples, validate on 8333 samples
Epoch 1/50
16667/16667 [==============================] - 25s 2ms/step - loss: 0.5003 - acc: 0.5264 - val_loss: 0.4272 - val_acc: 0.6019
Epoch 2/50
16667/16667 [==============================] - 24s 1ms/step - loss: 0.3581 - acc: 0.6526 - val_loss: 0.4040 - val_acc: 0.6408
Epoch 3/50
16667/16667 [==============================] - 25s 1ms/step - loss: 0.3248 - acc: 0.6860 - val_loss: 0.3899 - val_acc: 0.6580
Epoch 4/50
16667/16667 [==============================] - 25s 1ms/step - loss: 0.3044 - acc: 0.7049 - val_loss: 0.3836 - val_acc: 0.6678
Epoch 5/50
16667/16667 [==============================] - 25s 1ms/step - loss: 0.2910 - acc: 0.7209 - val_loss: 0.3785 - val_acc: 0.6771
Epoch 6/50
16667/16667 [==============================] - 24s 1ms/step - loss: 0.2809 - acc: 0.7293 - val_loss: 0.3754 - val_acc: 0.6840
Epoch 7/50
16667/16667 [==============================] - 25s 1ms/step - loss: 0.2727 - acc: 0.7354 - val_loss: 0.3733 - val_acc:

In [48]:
combined_model_history = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], batch_size=1, epochs=50)

Epoch 1/50
16667/16667 [==============================] - 53s 3ms/step - loss: 0.9080 - acc: 0.2106
Epoch 2/50
16667/16667 [==============================] - 51s 3ms/step - loss: 0.7730 - acc: 0.3036
Epoch 3/50
16667/16667 [==============================] - 51s 3ms/step - loss: 0.7338 - acc: 0.3283
Epoch 4/50
16667/16667 [==============================] - 52s 3ms/step - loss: 0.7127 - acc: 0.3405
Epoch 5/50
16667/16667 [==============================] - 51s 3ms/step - loss: 0.6986 - acc: 0.3467
Epoch 6/50
16667/16667 [==============================] - 51s 3ms/step - loss: 0.6885 - acc: 0.3530
Epoch 7/50
16667/16667 [==============================] - 52s 3ms/step - loss: 0.6808 - acc: 0.3590
Epoch 8/50
16667/16667 [==============================] - 52s 3ms/step - loss: 0.6745 - acc: 0.3619
Epoch 9/50
16667/16667 [==============================] - 52s 3ms/step - loss: 0.6692 - acc: 0.3644
Epoch 10/50
16667/16667 [==============================] - 52s 3ms/step - loss: 0.6648 - acc: 0.3664

In [49]:
# base_model_history_trainfalse = base_model_history
# combined_model_history_trainfalse = combined_model_history

In [50]:
net_weights = combined_model.get_weights()

In [51]:
# Evaluate model

### Generate color weighted

In [52]:
import re

class ColoredWeightedDoc(object):
    def __init__(self, doc, human_terms, ht_weights, token_pattern=r"(?u)\b\w\w+\b", binary = False):
        self.doc = doc
        self.human_terms = human_terms
        self.ht_weights = ht_weights
        self.binary = binary
        self.tokenizer = re.compile(token_pattern)
#         self.abs_ranges = np.linspace(0, max([abs(coefs.min()), abs(coefs.max())]), 8)
    def _repr_html_(self):
        html_rep = ""
        tokens = self.doc.split(" ") 
        if self.binary:
            seen_tokens = set()       
        for token in tokens:
            vocab_tokens = self.tokenizer.findall(token.lower())
            if len(vocab_tokens) > 0:
                vocab_token = vocab_tokens[0]
                try:
                    vocab_index = self.human_terms.index(vocab_token)
                    
                    if not self.binary or vocab_index not in seen_tokens:
                        
                        if self.ht_weights[vocab_index] == 0: # human-terms which has been washed out (opposing)
                            html_rep = html_rep + "<font size = 2, color=lightgreen> " + token + " </font>"
                        
                        elif self.ht_weights[vocab_index] != 0: # human-terms transparency
                            html_rep = html_rep + "<font size = 3, color=blue> " + token + " </font>"
                        
                        else: # neutral word
                            html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                        
                        if self.binary:    
                            seen_tokens.add(vocab_index)
                    
                    else: # if binary and this is a token we have seen before
                        html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                except: # this token does not exist in the vocabulary
                    html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
            else:
                html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
        return html_rep

In [53]:
from IPython import display
idx = 1001
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  0 
predict from base model :  [-1.] 
predict label :  [6.086446e-08]

awful 2.4234865
boring 2.5078313
horrible 2.1208763
pathetic 1.8960953
ridiculous 2.0476725
stupid 2.2427604
waste 2.3993704


In [54]:
np.where(np.sum(y_test_agreement, axis=1)!=0)[0][1000:1010]

array([1231, 1232, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241],
      dtype=int64)

In [55]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 2s 66us/step


[0.13470757930457591, 0.82444]

In [56]:
concat_all = concat_after_relu.predict([X_test, y_test_agreement])

In [57]:
concat_all.shape

(25000, 83)

In [58]:
indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

21916

In [59]:
not_zero_predict = combined_model.evaluate([X_test[indices], y_test_agreement[indices]], y_test_original[indices])

21916/21916 [==============================] - 1s 53us/step


In [60]:
not_zero_predict

[0.09944458844367605, 0.8789468881400272]